In [19]:
urls = ['http://httpbin.org/headers', 'http://httpbin.org/get', 'http://httpbin.org/user-agent']


### synchronous

In [31]:
import requests

def hello(urls):
    return [requests.get(url).content for url in urls]

%time hello(urls)

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 1.03 s


[b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.10.0"\n  }\n}\n',
 b'{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.10.0"\n  }, \n  "origin": "89.175.96.54", \n  "url": "http://httpbin.org/get"\n}\n',
 b'{\n  "user-agent": "python-requests/2.10.0"\n}\n']

### async

In [27]:
#!/usr/local/bin/python3.5

import asyncio
from aiohttp import ClientSession


async def fetch(url, session):
    async with session.get(url) as response:
        # async operation must be preceded by await,  to iterate over generator function
        return await response.read()

    
async def run(loop, urls):
    tasks = []
    
    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    async with ClientSession() as session:
        for url in urls:
            task = asyncio.ensure_future(fetch(url, session))
            tasks.append(task)
            
        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
        return responses
    

loop = asyncio.get_event_loop()

# 1 task
#%time loop.run_until_complete(hello())

# many tasks
future = asyncio.ensure_future(run(loop, urls))
%time loop.run_until_complete(future)

CPU times: user 0 ns, sys: 10 ms, total: 10 ms
Wall time: 372 ms


[b'{\n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "0", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.5 aiohttp/1.0.2"\n  }\n}\n',
 b'{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "0", \n    "Host": "httpbin.org", \n    "User-Agent": "Python/3.5 aiohttp/1.0.2"\n  }, \n  "origin": "89.175.96.54", \n  "url": "http://httpbin.org/get"\n}\n',
 b'{\n  "user-agent": "Python/3.5 aiohttp/1.0.2"\n}\n']